In [30]:
# Import dependencies
import pandas
import mechanize
import re
from bs4 import BeautifulSoup
from dateutil import rrule
from datetime import datetime, timedelta

In [35]:
# CONFIGURATION

# Date range for search.
# This program scrapes results month-by-month
from_date = datetime(1990,01,01)
to_date = datetime.today()

In [32]:
# HELPERS

deficiency_row_details_re = re.compile('Item No.:\s*(?P<item_number>\d{2})\s*Site:\s*(?P<site>.*)\s*Violation:\s*(?P<violation>.*)\s*Recommendation:\s*(?P<recommendation>.*)')
deficiency_summary_info_re = re.compile('Cruise Ship:\s*(?P<cruise_ship>.*)\s*Cruise Line:\s*(?P<cruise_line>.*)\s*Inspection Date:\s*(?P<date>.*)\s*Inspection Score:\s*(?P<score>.*)')

def row_values_to_dictionary(row_values):
    dictionary = {}
    dictionary['cruise_ship'] = row_values[0].get_text().strip(' \t\n\r')
    dictionary['cruise_line'] = row_values[1].get_text().strip(' \t\n\r')
    dictionary['date'] = row_values[2].get_text().strip(' \t\n\r')
    dictionary['score'] = row_values[3].get_text().strip(' \t\n\r')
    # dictionary['report_url'] = row_values[5].find('a')['href']
    return dictionary

def soup_to_list_of_dictionaries(soup):
    data_to_return = {'summaries':[], 'deficiencies':[]}
    table = soup.find('table',id='ctl00_ContentPlaceHolder1_dgMasterList')
    table_rows = table.find_all('tr')

    for row in table_rows:
        values = row.find_all('td')
        # If the row looks like a row of data, add it to the list
        if(len(values) == 7 and values[0].get_text() != "Cruise Ship"):
            data_to_return['summaries'].append(row_values_to_dictionary(values))
            data_to_return['deficiencies'].extend(get_list_of_deficiencies_for_inspection(browser))
    return data_to_return

def get_inspections_between(start_date, end_date):
    start_date_str = start_date.strftime('%m/%d/%Y')
    end_date_str = end_date.strftime('%m/%d/%Y')
    
    print "INFO :: {} => {} :: About to scrape inspections".format(
        start_date_str, end_date_str)
    
    # create mechanize browser and go to search form page
    browser.open('https://wwwn.cdc.gov/InspectionQueryTool/InspectionSearch.aspx')
    browser.select_form(name='aspnetForm')
    
    # set form input values and submit form
    browser['ctl00$ContentPlaceHolder1$rb_InspectionDateCriteria'] = ['rb_BetweenDate']
    browser['ctl00$ContentPlaceHolder1$txtFromSearchDate'] = start_date_str
    browser['ctl00$ContentPlaceHolder1$txtToSearchDate'] = end_date_str
    browser.submit()

    # read form response (resulting HTML page) and parse the data
    b = browser.response()
    soup = BeautifulSoup(b.read())
    data_for_range = soup_to_list_of_dictionaries(soup)
    print "INFO :: {} => {} :: {} inspections, {} deficiencies".format(
        start_date_str, end_date_str,
        len(data_for_range['summaries']), len(data_for_range['deficiencies']))
    return data_for_range

def get_list_of_deficiencies_for_inspection(browser):
    deficiencies_to_return = []
    for link in browser.links():
        if link.text == "Report":
            browser.follow_link(link)
            report_page_soup = BeautifulSoup(browser.response().read())
            deficiencies = report_page_soup.find_all("table",class_="tableCellBorderNone")
            summary_info = deficiency_summary_info_re.search(deficiencies[0].get_text())
            if summary_info: 
                for deficiency_row in deficiencies[2:]:
                    deficiency_row_details = deficiency_row_details_re.search(deficiency_row.get_text())
                    if deficiency_row_details:
                        deficiencies_to_return.append({
                            'cruise_ship': summary_info.group('cruise_ship').strip(' \t\n\r'),
                            'cruise_line': summary_info.group('cruise_line').strip(' \t\n\r'),
                            'inspection_score': summary_info.group('score').strip(' \t\n\r'),
                            'inspection_date': summary_info.group('date').strip(' \t\n\r'),
                            'deficiency_item_number': deficiency_row_details.group('item_number').strip(' \t\n\r'),
                            'deficiency_site': deficiency_row_details.group('site').strip(' \t\n\r'),
                            'deficiency_violation': deficiency_row_details.group('violation').strip(' \t\n\r'),
                            'deficiency_recommendation': deficiency_row_details.group('recommendation').strip(' \t\n\r')
                        })
    return deficiencies_to_return


In [38]:
# GET THE DATA! ( by month :-\ )
print datetime.now()

browser = mechanize.Browser()

summaries = []
deficiencies = []

# Month-looping code adapted from StackOverflow answer
months_within_range = list(rrule.rrule(rrule.MONTHLY, dtstart=from_date).between(from_date, to_date, inc=True))

i = 0
while i < len(months_within_range) - 1:
    month_start = months_within_range[i]
    month_end = months_within_range[i+1] - timedelta(days=1)
    data_for_period = get_inspections_between(month_start,month_end) 
    summaries.extend(data_for_period['summaries'])
    deficiencies.extend(data_for_period['deficiencies'])
    i += 1

print datetime.now()

print len(summaries)
print len(deficiencies)

2016-02-16 15:35:06.152858
INFO :: 01/01/1990 => 01/31/1990 :: About to scrape inspections
INFO :: 01/01/1990 => 01/31/1990 :: 0 inspections, 0 deficiencies
INFO :: 02/01/1990 => 02/28/1990 :: About to scrape inspections
INFO :: 02/01/1990 => 02/28/1990 :: 1 inspections, 0 deficiencies
INFO :: 03/01/1990 => 03/31/1990 :: About to scrape inspections
INFO :: 03/01/1990 => 03/31/1990 :: 2 inspections, 13 deficiencies
INFO :: 04/01/1990 => 04/30/1990 :: About to scrape inspections
INFO :: 04/01/1990 => 04/30/1990 :: 1 inspections, 0 deficiencies
INFO :: 05/01/1990 => 05/31/1990 :: About to scrape inspections
INFO :: 05/01/1990 => 05/31/1990 :: 0 inspections, 0 deficiencies
INFO :: 06/01/1990 => 06/30/1990 :: About to scrape inspections
INFO :: 06/01/1990 => 06/30/1990 :: 0 inspections, 0 deficiencies
INFO :: 07/01/1990 => 07/31/1990 :: About to scrape inspections
INFO :: 07/01/1990 => 07/31/1990 :: 1 inspections, 0 deficiencies
INFO :: 08/01/1990 => 08/31/1990 :: About to scrape inspection

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [41]:
# DO STUFF WITH THE DATA 

# First things first, convert the dictionary to a pandas dataframe
summaries_df = pandas.DataFrame.from_dict(summaries)
summaries_df = summaries_df.dropna(how='all') # remove empty rows
summaries_df.to_csv('summaries-19900101thru20160131.csv',index=False,encoding='utf-8')
# print summaries_df.to_csv()


deficiencies_df = pandas.DataFrame.from_dict(deficiencies)
deficiencies_df = deficiencies_df.dropna(how='all') # remove empty rows
deficiencies_df.to_csv('deficiencies-19900101thru20160131.csv',index=False,encoding='utf-8')
# print deficiencies_df.to_csv()


2016-02-16 16:34:53.366769


In [ ]:
# Questions? Happy to help.. just email mark.silverberg@socrata.com